In [1]:
import glob
import os
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('../dcm_download.csv',header=None)
df.head(),df.shape
df.columns = ['file']

In [2]:
df['institutionName'] = df['file'].apply(lambda x:x.split('_')[0])
df['accessionNumber'] = df['file'].apply(lambda x:x.split('_')[1])
df['seriesNumber']=df['file'].apply(lambda x:int(x.split('_')[2]))
df['seriesNumber'] = df['seriesNumber'].astype(str)

In [3]:
#jmid nii 対応表

jmid_nii = pd.read_excel('../JMID_NII施設ID対応表.xlsx')
jmid_nii['NII側ID'] = jmid_nii['NII側ID'].fillna(0).astype(int).astype(str)
jmid_nii['JMID側ID'] = jmid_nii['JMID側ID'].fillna(0).astype(int).astype(str)
#df['施設コード'] = df['施設コード'].astype(str)
df = df.merge(jmid_nii,left_on='institutionName',right_on='NII側ID',how='left')

In [4]:
df.columns = ['file', 'nii_institutionName', 'accessionNumber', 'seriesNumber', 'institutionName','NII側ID', '大学名']

In [123]:
tmp = pd.read_csv(csv_path,low_memory=False)
tmp['institutionName']

0       3319800011
1       3319800011
2       3319800011
3       3319800011
4       3319800011
           ...    
6003    3319800011
6004    3319800011
6005    3319800011
6006    3319800011
6007    3319800011
Name: institutionName, Length: 6008, dtype: int64

In [5]:
csv_paths = glob.glob('../jmid_cloud/*/*.csv')
cloud_df = pd.DataFrame()
for csv_path in tqdm(csv_paths):
    tmp = pd.read_csv(csv_path,low_memory=False)
    tmp['seriesNumber'] = tmp['seriesNumber'].astype(str)
    tmp['accessionNumber']=tmp['accessionNumber'].astype(str)
    tmp['institutionName'] = tmp['institutionName'].astype(str)    #print(tmp.shape)
    cloud_df = pd.concat([cloud_df,tmp])
cloud_df = cloud_df.reset_index(drop=True)

100%|██████████| 1031/1031 [16:42<00:00,  1.03it/s]


In [104]:
cloud_df.head()

,requestId,studyInstanceUID,seriesInstanceUID,r_reportId,r_clinicalReportId,studyGroupId,seriesNumber,modality,fov,bodyPartExamined,...,r_patientComments,r_patientOrganizationId,r_medicalHistory,r_readingRequestInfo,r_emergencyReadingInfo,r_reportFormatInfo,r_sharedMemoInfo,r_findingsStatus,r_subjectToAdditionalManagement,r_otherDocumentInfo
0,efb97e1e0ceb4a83ab80feaf89cb1488,1.2.392.200001.2.3.5.713.4.99.4.5134745420220824,1.2.392.200036.9116.2.6.1.54365.1690115318.166...,NaN,NaN,240001,1,CT,552.0x1492.0,CHEST_ABDOMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,efb97e1e0ceb4a83ab80feaf89cb1488,1.2.392.200001.2.3.5.713.4.99.4.5134745420220824,1.2.392.200036.9116.2.6.1.54365.1690115318.166...,NaN,NaN,240001,2,CT,349.2x349.2,CHEST_ABDOMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,efb97e1e0ceb4a83ab80feaf89cb1488,1.2.392.200001.2.3.5.713.4.99.4.5134745420220824,1.2.392.200036.9116.2.6.1.54365.1690115318.166...,NaN,NaN,240001,3,CT,349.2x349.2,CHEST_ABDOMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,efb97e1e0ceb4a83ab80feaf89cb1488,1.2.392.200001.2.3.5.713.4.99.4.5134745420220824,1.2.392.200036.9116.2.6.1.54365.1690115318.166...,NaN,NaN,240001,9001,SR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,efb97e1e0ceb4a83ab80feaf89cb1488,1.2.840.113619.2.416.2379293900273822345499379...,1.2.840.113619.2.416.1212020214188956660771399...,NaN,NaN,240002,2,CT,350.0x350.0,ABDOMEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
cloud_df['accessionNumber'] = cloud_df['accessionNumber'].apply(lambda x:x.split('.')[0])
cloud_df['institutionName'] = cloud_df['institutionName'].apply(lambda x:x.split('.')[0])

In [106]:
cloud_df['institutionName']

0          1318814790
1          1318814790
2          1318814790
3          1318814790
4          1318814790
              ...    
4492678    3319800011
4492679    3319800011
4492680    3319800011
4492681    3319800011
4492682    3319800011
Name: institutionName, Length: 4492683, dtype: object

In [107]:
merge_df = df.merge(cloud_df,how='inner',on=['accessionNumber','institutionName','seriesNumber'])

In [111]:
merge_df.drop_duplicates(subset='file').shape

(22472, 154)

In [116]:
merge_df[['requestId','studyInstanceUID','seriesInstanceUID','r_reportId','r_clinicalReportId']].isna().sum(axis=0)

requestId                 0
studyInstanceUID          0
seriesInstanceUID         0
r_reportId            19387
r_clinicalReportId    19387
dtype: int64

In [125]:
merge_df['accessionNumber']=merge_df['accessionNumber'].astype(int)
merge_df['institutionName']=merge_df['institutionName'].astype(int)
merge_df['seriesNumber'] =merge_df['seriesNumber'].astype(int)
merge_df[cloud_df.columns].iloc[:500].to_csv('merge_df.csv',index=False,encoding="utf_8_sig")

In [51]:
df['accessionNumber']

0         1210170648888928
1         1210170648888928
2         1210170749729057
3         1210170749729057
4         1210171152422457
                ...       
270664    2020102068446994
270665    2020102068446994
270666    2020102068446994
270667    2020102068459766
270668    2020102068459766
Name: accessionNumber, Length: 270669, dtype: object

In [52]:
cloud_df['accessionNumber']

0          5134745420220824.0
1          5134745420220824.0
2          5134745420220824.0
3          5134745420220824.0
4          5074099620220729.0
                  ...        
4492678      1232306320210207
4492679      1232306320210207
4492680      1232306320210207
4492681      1232306320210207
4492682      1232306320210207
Name: accessionNumber, Length: 4492683, dtype: object

In [50]:
df['seriesNumber']

0         2
1         3
2         2
3         3
4         3
         ..
270664    4
270665    5
270666    6
270667    8
270668    9
Name: seriesNumber, Length: 270669, dtype: int64

In [49]:
cloud_df['seriesNumber'].value_counts()

1         933065
2         716565
3         477606
4         340299
5         287068
           ...  
20039          1
900001         1
900000         1
8500           1
8504           1
Name: seriesNumber, Length: 972, dtype: int64

In [47]:
cloud_df['institutionName'].astype(str).value_counts()

1318814790      921795
2619900042      782156
3319800011      728800
1310514836      567347
2719900249      505332
1310415067      476650
4019819897      283055
3618010015       97852
3818010047       76793
118010016        48824
3818010047.0      2503
3319800011.0      1463
1310514836.0        94
########            13
nan                  6
Name: institutionName, dtype: int64

In [48]:
df['institutionName'].value_counts()

1318814790    69126
3319800011    61343
2619900042    51661
2719900249    29181
1310415067    27506
1310514836    12562
4019819897    11704
3818010047     5994
3618010015     1592
Name: institutionName, dtype: int64